# CLIP fine-tuning for Egyptian dishes (Kaggle)

This notebook is designed to run in **Kaggle** after you push your repo to **GitHub**.

What it does:
1. Clone your GitHub repo
2. Install dependencies
3. Run sanity checks (GPU, dataset path, folder-per-class layout)
4. Train **frozen OpenCLIP + linear head**
5. Evaluate **trained head** + **zero-shot baseline**
6. Zip `artifacts/` for download

> Kaggle settings:
> - Turn **GPU** ON
> - Turn **Internet** ON (needed for `git clone` + pip)


## 0) Set your GitHub repo

Replace `GITHUB_REPO` with your repository HTTPS URL.

If your training script lives in a subfolder, set `PROJECT_SUBDIR`.


In [ ]:
GITHUB_REPO = "https://github.com/<YOUR_USERNAME>/<YOUR_REPO>.git"
BRANCH = "main"
REPO_DIR = "repo"

# If your script is inside a subfolder (e.g. "clip_finetune"), set it here.
# Leave as "" if you run from repo root.
PROJECT_SUBDIR = ""  # example: "clip_finetune"


## 1) Clone repo

In [ ]:
!rm -rf {REPO_DIR}
!git clone --depth 1 --branch {BRANCH} {GITHUB_REPO} {REPO_DIR}

%cd {REPO_DIR}

import os, sys
if PROJECT_SUBDIR:
    os.chdir(PROJECT_SUBDIR)
    print("Changed directory to:", os.getcwd())

# Make current project directory importable
sys.path.append(os.getcwd())
print("PYTHONPATH +:", os.getcwd())


## 2) Install dependencies

If `requirements.txt` exists in the current directory, this installs it.
Otherwise installs minimal deps for OpenCLIP fine-tuning.


In [ ]:
import os

if os.path.exists("requirements.txt"):
    !pip -q install -r requirements.txt
else:
    !pip -q install open_clip_torch torch torchvision pillow numpy tqdm

print("Done installing dependencies")


## 3) Sanity checks

This checks:
- GPU availability
- Torch/CUDA versions
- Dataset path exists
- Dataset layout is folder-per-class
- Total image count


In [ ]:
from pathlib import Path
import torch

print("cwd:", Path.cwd())
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("cuda device:", torch.cuda.get_device_name(0))

# TODO: set this to your Kaggle dataset folder.
# It MUST contain subfolders = class names.
DATA_ROOT = "/kaggle/input/<YOUR_DATASET_NAME>/Egyptian"

root = Path(DATA_ROOT)
print("DATA_ROOT:", root)
print("exists:", root.exists())

if not root.exists():
    raise FileNotFoundError("DATA_ROOT does not exist. Update DATA_ROOT.")

class_dirs = [p.name for p in root.iterdir() if p.is_dir()]
class_dirs.sort()
print("num classes:", len(class_dirs))
print("sample classes:", class_dirs[:20])

exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp"}
num_imgs = sum(1 for p in root.rglob("*") if p.is_file() and p.suffix.lower() in exts)
print("total images:", num_imgs)

if len(class_dirs) == 0:
    raise ValueError("No class folders found. Expected folder-per-class dataset.")
if num_imgs == 0:
    raise ValueError("No images found under DATA_ROOT.")


## 4) Train + Evaluate (single script)

This runs your script (with tqdm logging).

Outputs:
- `artifacts/ckpts/egypt_clip_linear.pt`
- `artifacts/reports/metrics.json`


In [ ]:
import os, sys, subprocess
from pathlib import Path

# Adjust if you placed the script elsewhere.
SCRIPT = Path("clip_finetune/scripts/train_eval_clip_linear.py")
if not SCRIPT.exists():
    SCRIPT = Path("scripts/train_eval_clip_linear.py")

if not SCRIPT.exists():
    raise FileNotFoundError("Could not find train_eval_clip_linear.py. Update SCRIPT path in this cell.")

env = os.environ.copy()
env["PYTHONPATH"] = str(Path(".").resolve())

cmd = [
    sys.executable, str(SCRIPT),
    "--data_root", DATA_ROOT,
    "--device", "cuda",
    "--use_amp",
    "--epochs", "10",
    "--batch_size", "64",
    "--num_workers", "2",
    "--model_name", "ViT-B-32-quickgelu",
    "--pretrained", "openai",
]

print("Running:\n", " ".join(cmd))
subprocess.check_call(cmd, env=env)


## 5) View metrics

In [ ]:
import json
from pathlib import Path

metrics_path = Path("artifacts/reports/metrics.json")
print("metrics_path:", metrics_path.resolve())
print("exists:", metrics_path.exists())

if metrics_path.exists():
    metrics = json.loads(metrics_path.read_text(encoding="utf-8"))
    metrics


## 6) Zip artifacts for download

After this cell, the zip will appear in Kaggle's **Output** panel.


In [ ]:
!ls -la artifacts || true
!zip -r artifacts.zip artifacts || true
!ls -la artifacts.zip || true
